### 1) Write a Python script that fetches holiday data and saves it as a csv file named "holidays.csv".​ To obtain the data, you can either screen scrape the Time and Date​ site, or parse the API (free trial​ account required).

In [77]:
import pandas as pd
import numpy as np
import requests
import lxml.html
import datetime

In [78]:
#pulls the web page as a string
page = requests.get("https://www.timeanddate.com/holidays/us/")

In [79]:
# creates an XML tree out of it so , it can be used with xpath
tree = lxml.html.fromstring(page.text)

In [80]:
#Getting the Dates
dates = tree.xpath("//th/text()")[5:]
len(dates)

403

In [81]:
#Getting the Days
days = tree.xpath("//tr/td[1]/text()")[:403]
len(days)

403

In [82]:
#Getting the Holiday names 
holiday_names = tree.xpath("//td/a/text()")
len(holiday_names)

403

In [83]:
#Getting the Holiday type
holiday_type = tree.xpath("//tr/td[3]/text()")
len(holiday_type)

403

In [84]:
#Getting the Details
t = tree.xpath("//tr/td[4]")
details = []
for elem in t:
    val = elem.xpath('text()')
    temp = ''
    temp1 = ''
    if(len(val)>1):
        if(val[0]=='All except '):
            temp1 = temp1 + 'All except '
            
            for x in range(1,len(val)+1):
                tt = elem.xpath("span["+str(x)+"]/text()")
                temp1 = temp1 + tt[0] + ','
            details.append([temp1])
        else:
            ff = elem.xpath("span/text()")
            for x in ff:
                temp = temp+x + ','
            details.append([temp])       
    else:
        details.append(val)
len(details)

403

In [85]:
holidays_df = pd.DataFrame({'Date': dates,
                            'Day': days,
                            'Name':holiday_names,
                            'Type':holiday_type,
                            'Details': details})

In [86]:
holidays_df['Details'] = holidays_df['Details'].apply(lambda x: ','.join(map(str, x)))

In [87]:
holidays_df.head()

,Date,Day,Name,Type,Details
0,Jan 1,Tuesday,New Year's Day,Federal Holiday,
1,Jan 4,Friday,World Braille Day,Worldwide observance,
2,Jan 6,Sunday,Epiphany,Christian,
3,Jan 7,Monday,Orthodox Christmas Day,Orthodox,
4,Jan 7,Monday,International Programmers' Day,Worldwide observance,


In [88]:
holidays_df.to_csv("holidays.csv",index=False)

### 2) Using the csv file from 1), write a​ Python​ function that returns a list of dictionaries of the next 10 upcoming U.S. holidays and dates. 

E.g.


def holidays​():​

    pass



holidays​()​

[
    {
        "name​": "Bastille Day​",
        "date": "Sunday, July 14, 2019",
        "type": "Observance",
        "details​": ""
    },
    ...
]



In [89]:
holidays = pd.read_csv("holidays.csv")
holidays.head(10)

,Date,Day,Name,Type,Details
0,Jan 1,Tuesday,New Year's Day,Federal Holiday,
1,Jan 4,Friday,World Braille Day,Worldwide observance,
2,Jan 6,Sunday,Epiphany,Christian,
3,Jan 7,Monday,Orthodox Christmas Day,Orthodox,
4,Jan 7,Monday,International Programmers' Day,Worldwide observance,
5,Jan 13,Sunday,Stephen Foster Memorial Day,Observance,
6,Jan 14,Monday,Orthodox New Year,Orthodox,
7,Jan 18,Friday,Lee-Jackson Day,State holiday,Virginia
8,Jan 19,Saturday,Robert E. Lee's Birthday,State holiday,Florida
9,Jan 19,Saturday,Confederate Heroes' Day,State holiday,Texas


In [90]:
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 403 entries, 0 to 402
Data columns (total 5 columns):
Date       403 non-null object
Day        403 non-null object
Name       403 non-null object
Type       403 non-null object
Details    396 non-null object
dtypes: object(5)
memory usage: 15.8+ KB


In [91]:
#remove the - in the Date Column
holidays["Date"] = holidays["Date"].str.replace("-"," ")

In [92]:
#Adding a Year column
holidays["Year"] = 2019

In [93]:
#Splitting the Date column into Month & Day
new = holidays["Date"].str.split(" ", expand = True)
holidays["Month"]= new[0] 
holidays["Dates"]= new[1]
holidays.drop(columns =["Date"], inplace = True)

In [94]:
#Converting the Month,Date,Year to One column and converting to Datetime object
holidays['FullDate'] = holidays['Dates'].map(str) + '-' + holidays['Month'].map(str) + '-' + holidays['Year'].map(str)
holidays['FullDate']= pd.to_datetime(holidays['FullDate'])
holidays.head()

,Day,Name,Type,Details,Year,Month,Dates,FullDate
0,Tuesday,New Year's Day,Federal Holiday,,2019,Jan,1,2019-01-01
1,Friday,World Braille Day,Worldwide observance,,2019,Jan,4,2019-01-04
2,Sunday,Epiphany,Christian,,2019,Jan,6,2019-01-06
3,Monday,Orthodox Christmas Day,Orthodox,,2019,Jan,7,2019-01-07
4,Monday,International Programmers' Day,Worldwide observance,,2019,Jan,7,2019-01-07


In [95]:
#Converting Dataframe to a list of dictionaries based on the records
dict_holidays = holidays.to_dict('records')
dict_holidays[:5]

[{'Day': 'Tuesday',
  'Name': "New Year's Day",
  'Type': 'Federal Holiday',
  'Details': '\xa0',
  'Year': 2019,
  'Month': 'Jan',
  'Dates': '1',
  'FullDate': Timestamp('2019-01-01 00:00:00')},
 {'Day': 'Friday',
  'Name': 'World Braille Day',
  'Type': 'Worldwide observance',
  'Details': '\xa0',
  'Year': 2019,
  'Month': 'Jan',
  'Dates': '4',
  'FullDate': Timestamp('2019-01-04 00:00:00')},
 {'Day': 'Sunday',
  'Name': 'Epiphany',
  'Type': 'Christian',
  'Details': '\xa0',
  'Year': 2019,
  'Month': 'Jan',
  'Dates': '6',
  'FullDate': Timestamp('2019-01-06 00:00:00')},
 {'Day': 'Monday',
  'Name': 'Orthodox Christmas Day',
  'Type': 'Orthodox',
  'Details': '\xa0',
  'Year': 2019,
  'Month': 'Jan',
  'Dates': '7',
  'FullDate': Timestamp('2019-01-07 00:00:00')},
 {'Day': 'Monday',
  'Name': "International Programmers' Day",
  'Type': 'Worldwide observance',
  'Details': '\xa0',
  'Year': 2019,
  'Month': 'Jan',
  'Dates': '7',
  'FullDate': Timestamp('2019-01-07 00:00:00')}]

In [96]:
def holiday_10():
    hols_list = []
    curr_date = datetime.datetime.today()
    print("List Of Holidays:\n")
    for hols in dict_holidays:
        if(curr_date <= hols['FullDate']):
            hols_list.append(hols)
              
    sorted_hols = sorted(hols_list, key=lambda x: x['FullDate'])
    print("\n")
    print(sorted_hols[:10]) 
        
holiday_10()

List Of Holidays:



[{'Day': 'Thursday', 'Name': 'Nelson Mandela Day', 'Type': 'United Nations observance', 'Details': '\xa0', 'Year': 2019, 'Month': 'Jul', 'Dates': '18', 'FullDate': Timestamp('2019-07-18 00:00:00')}, {'Day': 'Wednesday', 'Name': 'Pioneer Day', 'Type': 'State holiday', 'Details': 'Utah', 'Year': 2019, 'Month': 'Jul', 'Dates': '24', 'FullDate': Timestamp('2019-07-24 00:00:00')}, {'Day': 'Saturday', 'Name': 'National Korean War Veterans Armistice Day', 'Type': 'Observance', 'Details': '\xa0', 'Year': 2019, 'Month': 'Jul', 'Dates': '27', 'FullDate': Timestamp('2019-07-27 00:00:00')}, {'Day': 'Sunday', 'Name': 'World Hepatitis Day', 'Type': 'United Nations observance', 'Details': '\xa0', 'Year': 2019, 'Month': 'Jul', 'Dates': '28', 'FullDate': Timestamp('2019-07-28 00:00:00')}, {'Day': 'Sunday', 'Name': "Parents' Day", 'Type': 'Observance', 'Details': '\xa0', 'Year': 2019, 'Month': 'Jul', 'Dates': '28', 'FullDate': Timestamp('2019-07-28 00:00:00')}, {'Day': 'Tuesday', 'N

### 3) Add a keyword argument "holidayType​"​ to the function from 2) such that it only returns holidays matching the given type, 

e.g., 


def holidays​(holidayType​​=None​):​

    pass

holidays​​(holidayType​​​="federal")    

[
   {
       "name​": "Labor​ Day​",
       "date": "Monday​, September 2, 2019",
       "type": "Federal Holiday​",
       "details​": ""
   },
   ...
]

In [97]:
def holiday_type_func(holiday_type=None):
    hols_list = []
    curr_date = datetime.datetime.today()
    print("List Of Holidays:\n")
    for hols in dict_holidays:
        if(curr_date <= hols['FullDate']):
            if(hols['Type'] == holiday_type):
                hols_list.append(hols)
            elif(hols['Type'] == holiday_type):
                hols_list.append(hols)
              
    sorted_hols = sorted(hols_list, key=lambda x: x['FullDate'])
    print("\n")
    print(sorted_hols) 
    return

holiday_type_func("Federal Holiday")

List Of Holidays:



[{'Day': 'Monday', 'Name': 'Labor Day', 'Type': 'Federal Holiday', 'Details': '\xa0', 'Year': 2019, 'Month': 'Sep', 'Dates': '2', 'FullDate': Timestamp('2019-09-02 00:00:00')}, {'Day': 'Monday', 'Name': 'Columbus Day', 'Type': 'Federal Holiday', 'Details': 'All except AK,AR,CA,DE,FL,HI,MI,MN,ND,NM,NV,OR,SD,TX,VT,WA,WY,', 'Year': 2019, 'Month': 'Oct', 'Dates': '14', 'FullDate': Timestamp('2019-10-14 00:00:00')}, {'Day': 'Monday', 'Name': 'Veterans Day', 'Type': 'Federal Holiday', 'Details': '\xa0', 'Year': 2019, 'Month': 'Nov', 'Dates': '11', 'FullDate': Timestamp('2019-11-11 00:00:00')}, {'Day': 'Thursday', 'Name': 'Thanksgiving Day', 'Type': 'Federal Holiday', 'Details': '\xa0', 'Year': 2019, 'Month': 'Nov', 'Dates': '28', 'FullDate': Timestamp('2019-11-28 00:00:00')}, {'Day': 'Wednesday', 'Name': 'Christmas Day', 'Type': 'Federal Holiday', 'Details': '\xa0', 'Year': 2019, 'Month': 'Dec', 'Dates': '25', 'FullDate': Timestamp('2019-12-25 00:00:00')}]
